In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # nopep8
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION"] = '1'

FILE_PATH = os.getcwd()
import sys
sys.path.append('\\'.join(FILE_PATH.split('\\')[:3]))
import argparse
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import pandas as pd
from trainvaltest import trainvaltest

tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

Amount of labels -> 101 

CricketShot has the most samples with 167 clips
PlayingViolin has the least samples with 100 clips

All actions have: 25 samples

Top 5 actions with least clips per sample:
                Total Clips  Samples  Clips per sample
Labels                                               
PlayingViolin          100       25              4.00
PullUps                100       25              4.00
Skijet                 100       25              4.00
TaiChi                 100       25              4.00
BreastStroke           101       25              4.04 

Top 5 actions with most clips per sample
               Total Clips  Samples  Clips per sample
Labels                                              
CricketShot           167       25              6.68
TennisSwing           166       25              6.64
HorseRiding           164       25              6.56
PlayingCello          164       25              6.56
PlayingDhol           164       25              6.56

train/

In [2]:
MODEL_NAME = "my_VGG19"
MODEL_PATH = f"{MODEL_NAME}"
BATCH_SIZE = 16
LABELS, INPUT_SHAPE, Train_Data, Val_Data, Test_Data = trainvaltest(BATCH_SIZE=BATCH_SIZE)
EPOCHS = 15
VERBOSE = 1


Train:
Found 386679 images belonging to 101 classes.
Validation:
Found 38626 images belonging to 101 classes.
Test:
Found 122374 images belonging to 101 classes.

Input shape -> (224, 224, 3)



In [3]:
class Preprocess(layers.Layer):
    def __init__(self, factor=0.2, scale=1.0 / 255.0, flipmode='horizontal', seed=182):
        """Image preprocessing layer Block

        Args:
            factor (float, optional): factor to set for rotation, brightness, zoom and image shifting. Defaults to 0.2.
            scale (float, optional): float to multiple all features by and normalize tensorflow. Defaults to 1.0/255.0.
            seed (int, optional): set the seed value for all preprocessing layers. Defaults to 182
        """
        super(Preprocess, self).__init__()
        self.factor = factor
        self.scale = scale
        self.seed = seed
        self.flipmode = flipmode

        self.rescale = preprocessing.Rescaling(scale=self.scale)
        self.randomrotate = preprocessing.RandomRotation(factor=self.factor, seed=self.seed)
        self.randomzoom = preprocessing.RandomZoom(height_factor=self.factor, width_factor=self.factor, seed=self.seed)
        self.shift = preprocessing.RandomTranslation(height_factor=self.factor, width_factor=self.factor, seed=self.seed)
        self.flip = preprocessing.RandomFlip(mode=self.flipmode, seed=self.seed)

    @tf.function
    def call(self, image):
        """apply all preprocessing steps in

        Args:
            image (tensor): numerical data of image

        Returns:
            tensor: preprocessed data
        """
        image = self.rescale(image)
        image = self.randomrotate(image)
        image = self.randomzoom(image)
        image = self.shift(image)
        image = self.flip(image)
        return image

In [4]:
class CNNBlock(layers.Layer):
    def __init__(self, filters, quad=False, conv_kernel_size=(3, 3), conv_strides=(1, 1), pool_size=(2, 2), pool_strides=(2, 2), padding='same'):
        """block of either double (or triple) conv layers

        Args:
            filters (int): numbers of filters for the conv layers within this block
            quad (bool, optional): whether this conv block contains double (2) or quadruple (4) conv layers. Defaults to False.
            conv_strides (tuple, optional): tuple to set strides value for conv layers. Defaults to (1, 1).
            conv_kernel_size (tuple, optional): kernel size for the conv layers in this block. Defaults to (3, 3).
            pool_size (tuple, optional): pool size for pooling layer for this block. Defaults to (2, 2).
            pool_strides (tuple, optional): strides value for pooling for this block. Defaults to (2, 2).
            padding (str, optional): padding value of conv layers. Defaults to 'same'.
        """
        super(CNNBlock, self).__init__()
        self.quad = quad
        self.pool_size = pool_size
        self.pool_strides = pool_strides
        self.filters = filters
        self.conv_kernel_size = conv_kernel_size
        self.conv_strides = conv_strides
        self.padding = padding

        self.conv1 = self.conv_layer()
        self.conv2 = self.conv_layer()
        if self.quad == True:
            self.conv3 = self.conv_layer()
            self.conv4 = self.conv_layer()
        self.batch_norm = layers.BatchNormalization()
        self.maxpooling = layers.MaxPooling2D(pool_size=self.pool_size, strides=self.pool_strides)

    def conv_layer(self):
        return layers.Conv2D(filters=self.filters, kernel_size=self.conv_kernel_size, strides=self.conv_strides,
                             padding=self.padding, activation=layers.ReLU())

    @tf.function
    def call(self, input_tensor, training=False):
        """forward propagation

        Args:
            input_tensor (input_tensor): input tensor for this data point
            training (bool): whether to set batch normalization to training or not

        Returns:
            tensor: output of the current CNN block
        """
        x = self.conv1(input_tensor)
        x = self.conv2(x)
        if self.quad == True:
            x = self.conv3(x)
            x = self.conv4(x)
        x = self.batch_norm(x, training=training)
        x = self.maxpooling(x)
        return x

In [5]:
class Model(keras.Model):
    def __init__(self, n_labels):
        """model build via subclassing

        Args:
            n_labels (int): amount of labels for the model to predict
        """
        super(Model, self).__init__()
        self.n_labels = n_labels

        self.preprocess = Preprocess()
        self.cnnblock1 = CNNBlock(filters=64)
        self.cnnblock2 = CNNBlock(filters=128)
        self.cnnblock3 = CNNBlock(filters=256, quad=True)
        self.cnnblock4 = CNNBlock(filters=512, quad=True)
        self.cnnblock5 = CNNBlock(filters=512, quad=True)
        self.globalmaxpooling = layers.GlobalMaxPooling2D()
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(4096, activation=layers.ReLU())
        self.fc2 = layers.Dense(4096, activation=layers.ReLU())
        self.outputs = layers.Dense(self.n_labels)

    @tf.function
    def call(self, input_tensor):
        """forward propagation for the entire model between each layer

        Args:
            input_tensor (tensor): output of the previous layer

        Returns:
            tensor: output of the previous tensor
        """
        x = self.preprocess(input_tensor)
        x = self.cnnblock1(x)
        x = self.cnnblock2(x)
        x = self.cnnblock3(x)
        x = self.cnnblock4(x)
        x = self.cnnblock5(x)
        x = self.globalmaxpooling(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.fc2(x)
        x = self.outputs(x)
        return x

In [6]:
def create_model(inp_shape, n_labels, model_name, layer_names):
    """creates model (input and output), name layers and compile

    Args:
        inp_shape (tuple(int)): tuple of ints, input shape
        n_labels (int): number of labels for last layer
        model_name (str): name of model
        layer_names (list(str)): list of names for each layer in model

    Returns:
        model: named and configured model with input/output and named layers
    """
    model = keras.Sequential()
    for i, layer in enumerate(Model(n_labels=n_labels).layers):
        model.add(layer)
        layer._name = layer_names[i]
    
    model._name = model_name
    model.build(input_shape=(None, *inp_shape))

    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=[keras.metrics.SparseCategoricalAccuracy(name='acc')])

    model.summary()
    return model

In [7]:
layer_names = tuple([
    "Preprocessing",
    "CNNBlock64_Double",
    "CNNBlock128_Double",
    "CNNBlock256_Triple",
    "CNNBlock512_Triple_1",
    "CNNBlock512_Triple_2",
    "GlobalMaxPooling",
    "Flatten",
    "FC",
    "FC2",
    "Outputs"
])

In [8]:
model = create_model(inp_shape=INPUT_SHAPE, n_labels=LABELS, model_name=MODEL_NAME, layer_names=layer_names)
earlystopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=VERBOSE)
callbacks = [earlystopping]

best_checkpoint = keras.callbacks.ModelCheckpoint(filepath=MODEL_PATH,
                                                  monitor='val_acc',
                                                  save_best_only=True,
                                                  save_freq='epoch',
                                                  verbose=VERBOSE)
callbacks.append(best_checkpoint)

Model: "my_VGG19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Preprocessing (Preprocess)   multiple                  0         
_________________________________________________________________
CNNBlock64_Double (CNNBlock) multiple                  38976     
_________________________________________________________________
CNNBlock128_Double (CNNBlock multiple                  221952    
_________________________________________________________________
CNNBlock256_Triple (CNNBlock multiple                  2066432   
_________________________________________________________________
CNNBlock512_Triple_1 (CNNBlo multiple                  8261632   
_________________________________________________________________
CNNBlock512_Triple_2 (CNNBlo multiple                  9441280   
_________________________________________________________________
GlobalMaxPooling (GlobalMaxP multiple                  0  

In [9]:
train = model.fit(Train_Data,
                  epochs=EPOCHS,
                  verbose=VERBOSE,
                  steps_per_epoch=len(Train_Data) // BATCH_SIZE,
                  callbacks=callbacks,
                  validation_data=Val_Data,
                  validation_steps=len(Val_Data) // BATCH_SIZE,
                  use_multiprocessing=True,
                  workers=-1)

Epoch 1/15
1510/1510 [==============================] - 1896s 1s/step - loss: 3.8650 - acc: 0.1157 - val_loss: 3.3756 - val_acc: 0.1725

Epoch 00001: val_acc improved from -inf to 0.17250, saving model to my_VGG19


INFO:tensorflow:Assets written to: my_VGG19\assets


INFO:tensorflow:Assets written to: my_VGG19\assets


Epoch 2/15
1510/1510 [==============================] - 1949s 1s/step - loss: 3.0789 - acc: 0.2385 - val_loss: 2.7505 - val_acc: 0.3092

Epoch 00002: val_acc improved from 0.17250 to 0.30917, saving model to my_VGG19


INFO:tensorflow:Assets written to: my_VGG19\assets


...

Epoch 11/15
1510/1510 [==============================] - 1905s 1s/step - loss: 0.6765 - acc: 0.8094 - val_loss: 0.5645 - val_acc: 0.8346

Epoch 00011: val_acc improved from 0.80542 to 0.83458, saving model to my_VGG19


INFO:tensorflow:Assets written to: my_VGG19\assets


INFO:tensorflow:Assets written to: my_VGG19\assets


Epoch 12/15
1510/1510 [==============================] - 1863s 1s/step - loss: 0.5939 - acc: 0.8305 - val_loss: 0.6226 - val_acc: 0.8229

Epoch 00012: val_acc did not improve from 0.83458
Epoch 13/15
 172/1510 [==>...........................] - ETA: 25:58 - loss: 0.5459 - acc: 0.8510

KeyboardInterrupt: 

In [10]:
test = model.evaluate(Test_Data, steps=len(Test_Data) // BATCH_SIZE, workers=-1, use_multiprocessing=True, verbose=VERBOSE)

478/478 [==============================] - 471s 986ms/step - loss: 5.0522 - acc: 0.2282


In [ ]:
train_history = pd.DataFrame(train.history)
train_history.head()

In [ ]:
plt.figure(figsize=(8, 6))
plt.title("Training")
def plotline(label):
    return plt.plot(train_history[label], label=label, linestyle='--' if label[:3]=='val' else '-')

for label in train_history.keys():
    plotline(label=label)
plt.xlabel('Epochs')
plt.legend()
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()